# 第12章: クラスとオブジェクトの基礎 - 解答集

この解答集では、課題の解答と詳細な解説を提供します。

**重要**: 自分で解いてから解答を見るようにしてください！

---
## 課題1: 図書館管理システム（基礎レベル）- 解答

### 解答コード

In [ ]:
<?php

/**
 * Bookクラス - 本を表す
 */
class Book {
    private string $title;
    private string $author;
    private string $isbn;
    private bool $isBorrowed;
    
    public function __construct(string $title, string $author, string $isbn) {
        $this->title = $title;
        $this->author = $author;
        $this->isbn = $isbn;
        $this->isBorrowed = false;  // 初期状態は貸出可能
    }
    
    /**
     * 本を借りる
     * @return bool 成功したかどうか
     */
    public function borrow(): bool {
        if ($this->isBorrowed) {
            return false;  // 既に借りられている
        }
        $this->isBorrowed = true;
        return true;
    }
    
    /**
     * 本を返す
     */
    public function returnBook(): void {
        $this->isBorrowed = false;
    }
    
    /**
     * 本の情報を取得
     */
    public function getInfo(): string {
        $status = $this->isBorrowed ? "[貸出中]" : "[利用可能]";
        return "{$this->title} by {$this->author} (ISBN: {$this->isbn}) {$status}";
    }
    
    // ゲッター
    public function getTitle(): string {
        return $this->title;
    }
    
    public function getIsbn(): string {
        return $this->isbn;
    }
    
    public function isBorrowed(): bool {
        return $this->isBorrowed;
    }
}

/**
 * Libraryクラス - 図書館を表す
 */
class Library {
    private string $name;
    private array $books;  // ISBN => Book
    
    public function __construct(string $name) {
        $this->name = $name;
        $this->books = [];
    }
    
    /**
     * 本を追加
     */
    public function addBook(Book $book): void {
        $this->books[$book->getIsbn()] = $book;
    }
    
    /**
     * 本を削除
     * @return bool 成功したかどうか
     */
    public function removeBook(string $isbn): bool {
        if (!isset($this->books[$isbn])) {
            return false;
        }
        unset($this->books[$isbn]);
        return true;
    }
    
    /**
     * ISBNで本を検索
     */
    public function findBookByIsbn(string $isbn): ?Book {
        return $this->books[$isbn] ?? null;
    }
    
    /**
     * 全ての本をリスト表示
     */
    public function listBooks(): void {
        if (empty($this->books)) {
            echo "蔵書はありません\n";
            return;
        }
        
        foreach ($this->books as $book) {
            echo "  - " . $book->getInfo() . "\n";
        }
    }
    
    public function getName(): string {
        return $this->name;
    }
}

// 動作確認
$library = new Library("中央図書館");

$book1 = new Book("PHP入門", "山田太郎", "978-4-00000-000-1");
$book2 = new Book("Laravel実践", "鈴木花子", "978-4-00000-000-2");
$book3 = new Book("デザインパターン", "佐藤次郎", "978-4-00000-000-3");

$library->addBook($book1);
$library->addBook($book2);
$library->addBook($book3);

echo "=== {$library->getName()} の蔵書 ===\n";
$library->listBooks();

$searchBook = $library->findBookByIsbn("978-4-00000-000-1");
if ($searchBook && $searchBook->borrow()) {
    echo "\n" . $searchBook->getTitle() . "を借りました\n";
}

echo "\n現在の状態:\n";
$library->listBooks();

if (!$searchBook->borrow()) {
    echo "\n既に借りられています\n";
}

$searchBook->returnBook();
echo "\n" . $searchBook->getTitle() . "を返しました\n";
echo $searchBook->getInfo() . "\n";

?>

### 解説

#### 1. クラス設計の基本

**単一責任の原則**:
- `Book` クラスは本のデータと状態管理だけを担当
- `Library` クラスは本のコレクション管理を担当
- 各クラスが明確な役割を持つことでコードが理解しやすい

#### 2. 状態管理

**`isBorrowed` フラグ**:
```php
private bool $isBorrowed;

public function borrow(): bool {
    if ($this->isBorrowed) {
        return false;  // 既に借りられている
    }
    $this->isBorrowed = true;
    return true;
}
```

**ポイント**:
- 状態をチェックしてから変更する（バリデーション）
- 不正な状態遷移を防ぐ
- 成功/失敗を返り値で表現

#### 3. コレクション管理

**ISBNをキーにする利点**:
```php
$this->books[$book->getIsbn()] = $book;
```
- 高速な検索（O(1)）
- 重複を自動的に防ぐ
- 自然な一意性（ISBNは一意）

---
## 課題2: RPGゲームキャラクター（中級レベル）- 解答

### 解答コード

In [ ]:
<?php

/**
 * Characterクラス - RPGキャラクターを表す
 */
class Character {
    private string $name;
    private int $hp;
    private int $maxHp;
    private int $mp;
    private int $maxMp;
    private int $attack;
    private int $defense;
    private int $level;
    private int $experience;
    
    public function __construct(
        string $name,
        int $hp = 100,
        int $mp = 50,
        int $attack = 20,
        int $defense = 10
    ) {
        $this->name = $name;
        $this->maxHp = $hp;
        $this->hp = $hp;
        $this->maxMp = $mp;
        $this->mp = $mp;
        $this->attack = $attack;
        $this->defense = $defense;
        $this->level = 1;
        $this->experience = 0;
    }
    
    /**
     * 攻撃を行う
     * @return int 与えたダメージ
     */
    public function attack(Character $target): int {
        // ダメージ計算式: 攻撃力 - 防御力/2（最低1ダメージ）
        $defenseReduction = (int)($target->defense / 2);
        $damage = max(1, $this->attack - $defenseReduction);
        
        $target->takeDamage($damage);
        return $damage;
    }
    
    /**
     * ダメージを受ける
     */
    public function takeDamage(int $damage): void {
        $this->hp = max(0, $this->hp - $damage);
    }
    
    /**
     * 回復する
     */
    public function heal(int $amount): void {
        $this->hp = min($this->maxHp, $this->hp + $amount);
    }
    
    /**
     * 経験値を獲得する
     * @return bool レベルアップしたかどうか
     */
    public function gainExperience(int $exp): bool {
        $this->experience += $exp;
        $requiredExp = $this->level * 100;
        
        if ($this->experience >= $requiredExp) {
            $this->levelUp();
            return true;
        }
        return false;
    }
    
    /**
     * レベルアップ
     */
    private function levelUp(): void {
        $this->level++;
        $this->experience = 0;  // 繰り越さない仕様
        
        // ステータス上昇
        $this->maxHp += 20;
        $this->maxMp += 10;
        $this->attack += 5;
        $this->defense += 3;
        
        // 全回復
        $this->hp = $this->maxHp;
        $this->mp = $this->maxMp;
    }
    
    /**
     * 生存しているか
     */
    public function isAlive(): bool {
        return $this->hp > 0;
    }
    
    /**
     * ステータスを表示
     */
    public function getStatus(): string {
        $status = $this->isAlive() ? "[生存]" : "[死亡]";
        return sprintf(
            "%s Lv.%d HP: %d/%d MP: %d/%d 攻撃力: %d 防御力: %d %s",
            $this->name,
            $this->level,
            $this->hp,
            $this->maxHp,
            $this->mp,
            $this->maxMp,
            $this->attack,
            $this->defense,
            $status
        );
    }
    
    // ゲッター
    public function getName(): string { return $this->name; }
    public function getLevel(): int { return $this->level; }
    public function getHp(): int { return $this->hp; }
    public function getMaxHp(): int { return $this->maxHp; }
}

// 動作確認
$hero = new Character("勇者", 100, 50, 25, 15);
$monster = new Character("ドラゴン", 200, 30, 30, 20);

echo "=== 戦闘開始 ===\n";
echo $hero->getStatus() . "\n";
echo $monster->getStatus() . "\n";

echo "\n=== 1ターン目 ===\n";
$damage1 = $hero->attack($monster);
echo "{$hero->getName()}の攻撃！ {$monster->getName()}に{$damage1}のダメージ！\n";
echo $monster->getStatus() . "\n";

$damage2 = $monster->attack($hero);
echo "{$monster->getName()}の攻撃！ {$hero->getName()}に{$damage2}のダメージ！\n";
echo $hero->getStatus() . "\n";

echo "\n=== 回復 ===\n";
$hero->heal(30);
echo "{$hero->getName()}は回復呪文を唱えた！\n";
echo $hero->getStatus() . "\n";

echo "\n=== 経験値獲得 ===\n";
$leveledUp = $hero->gainExperience(150);
echo "{$hero->getName()}は150の経験値を獲得！\n";
if ($leveledUp) {
    echo "*** レベルアップ！ ***\n";
}
echo $hero->getStatus() . "\n";

?>

### 解説

#### 1. ゲームロジックの実装

**ダメージ計算**:
```php
$damage = max(1, $this->attack - $target->defense / 2);
```
- `max(1, ...)` で最低1ダメージを保証
- 防御力が高いほどダメージを軽減
- バランス調整が容易

#### 2. 状態のクランプ

**範囲外の値を防ぐ**:
```php
$this->hp = max(0, $this->hp - $damage);  // 0未満にならない
$this->hp = min($this->maxHp, $this->hp + $amount);  // maxHpを超えない
```

**`max()` と `min()` の使い方**:
- `max(0, $value)` - 下限を設定
- `min($max, $value)` - 上限を設定
- 組み合わせて範囲を制限

#### 3. レベルアップシステム

**経験値テーブル**:
```php
$requiredExp = $this->level * 100;
```

レベル | 必要経験値 | 累計
---|---|---
1 | 100 | 100
2 | 200 | 300
3 | 300 | 600
4 | 400 | 1000

**実務での応用**:
- ユーザーランクシステム
- ポイント制度
- 成長の可視化

#### 4. sprintf() による整形

```php
sprintf(
    "%s Lv.%d HP: %d/%d",
    $this->name,
    $this->level,
    $this->hp,
    $this->maxHp
);
```

**フォーマット指定子**:
- `%s` - 文字列
- `%d` - 整数
- `%f` - 浮動小数点
- `%.2f` - 小数点以下2桁

---
## 課題3: 銀行口座システム（上級レベル）- 解答

### 解答コード

In [ ]:
<?php

/**
 * Transactionクラス - 取引履歴を表す
 */
class Transaction {
    public const TYPE_DEPOSIT = 'deposit';
    public const TYPE_WITHDRAW = 'withdraw';
    public const TYPE_TRANSFER = 'transfer';
    
    private string $type;
    private int $amount;
    private string $description;
    private DateTime $timestamp;
    private int $balanceAfter;
    
    public function __construct(string $type, int $amount, string $description, int $balanceAfter) {
        $this->type = $type;
        $this->amount = $amount;
        $this->description = $description;
        $this->timestamp = new DateTime();
        $this->balanceAfter = $balanceAfter;
    }
    
    public function toArray(): array {
        return [
            'type' => $this->type,
            'amount' => $this->amount,
            'description' => $this->description,
            'timestamp' => $this->timestamp->format('Y-m-d H:i:s'),
            'balance_after' => $this->balanceAfter
        ];
    }
}

/**
 * BankAccountクラス - 銀行口座を表す
 */
class BankAccount {
    private string $accountNumber;
    private string $ownerName;
    private int $balance;
    private array $transactions;
    private DateTime $createdAt;
    
    public function __construct(string $accountNumber, string $ownerName, int $initialBalance = 0) {
        $this->accountNumber = $accountNumber;
        $this->ownerName = $ownerName;
        $this->balance = $initialBalance;
        $this->transactions = [];
        $this->createdAt = new DateTime();
        
        // 初預金も履歴に記録
        if ($initialBalance > 0) {
            $this->recordTransaction(
                new Transaction(
                    Transaction::TYPE_DEPOSIT,
                    $initialBalance,
                    "口座開設",
                    $this->balance
                )
            );
        }
    }
    
    /**
     * 入金
     */
    public function deposit(int $amount, string $description = "入金"): bool {
        if ($amount <= 0) {
            return false;
        }
        
        $this->balance += $amount;
        $this->recordTransaction(
            new Transaction(
                Transaction::TYPE_DEPOSIT,
                $amount,
                $description,
                $this->balance
            )
        );
        return true;
    }
    
    /**
     * 出金
     */
    public function withdraw(int $amount, string $description = "出金"): bool {
        if ($amount <= 0 || $amount > $this->balance) {
            return false;
        }
        
        $this->balance -= $amount;
        $this->recordTransaction(
            new Transaction(
                Transaction::TYPE_WITHDRAW,
                $amount,
                $description,
                $this->balance
            )
        );
        return true;
    }
    
    /**
     * 振込
     */
    public function transfer(BankAccount $toAccount, int $amount, string $description = "振込"): bool {
        // 自分の口座から出金
        if (!$this->withdraw($amount, $description)) {
            return false;
        }
        
        // 宛先口座に入金
        $toAccount->deposit($amount, $description);
        
        return true;
    }
    
    /**
     * 残高を取得
     */
    public function getBalance(): int {
        return $this->balance;
    }
    
    /**
     * 取引履歴を取得
     */
    public function getTransactionHistory(): array {
        return $this->transactions;
    }
    
    /**
     * 口座照会書を発行
     */
    public function getStatement(): string {
        $output = "";
        $output .= "========================================\n";
        $output .= "           口座照会書\n";
        $output .= "========================================\n";
        $output .= "口座番号: {$this->accountNumber}\n";
        $output .= "口座名義: {$this->ownerName}\n";
        $output .= "作成日: " . $this->createdAt->format('Y-m-d') . "\n";
        $output .= "発行日: " . (new DateTime())->format('Y-m-d H:i:s') . "\n";
        $output .= "----------------------------------------\n";
        $output .= "現在の残高: " . number_format($this->balance) . "円\n";
        $output .= "========================================\n";
        $output .= "           取引履歴\n";
        $output .= "========================================\n";
        
        foreach ($this->transactions as $i => $txn) {
            $data = $txn->toArray();
            $typeLabel = match($data['type']) {
                Transaction::TYPE_DEPOSIT => '入金',
                Transaction::TYPE_WITHDRAW => '出金',
                Transaction::TYPE_TRANSFER => '振込',
                default => $data['type']
            };
            $output .= sprintf(
                "%2d. [%s] %s %13s円 残高:%13s円 %s\n",
                $i + 1,
                $data['timestamp'],
                $typeLabel,
                number_format($data['amount']),
                number_format($data['balance_after']),
                $data['description']
            );
        }
        
        $output .= "========================================\n";
        return $output;
    }
    
    private function recordTransaction(Transaction $transaction): void {
        $this->transactions[] = $transaction;
    }
    
    public function getAccountNumber(): string { return $this->accountNumber; }
    public function getOwnerName(): string { return $this->ownerName; }
}

// 動作確認
$account1 = new BankAccount("123-456-789", "田中太郎", 100000);
$account2 = new BankAccount("987-654-321", "鈴木花子", 50000);

echo "=== 口座開設 ===\n";
echo "{$account1->getOwnerName()}の口座残高: " . number_format($account1->getBalance()) . "円\n";
echo "{$account2->getOwnerName()}の口座残高: " . number_format($account2->getBalance()) . "円\n";

echo "\n=== 入金 ===\n";
$account1->deposit(50000, "給料");
echo "{$account1->getOwnerName()}の口座残高: " . number_format($account1->getBalance()) . "円\n";

echo "\n=== 出金 ===\n";
$account1->withdraw(30000, "生活費");
echo "{$account1->getOwnerName()}の口座残高: " . number_format($account1->getBalance()) . "円\n";

echo "\n=== 振込 ===\n";
$success = $account1->transfer($account2, 25000, "家賃");
if ($success) {
    echo "振込成功！\n";
} else {
    echo "振込失敗（残高不足）\n";
}

echo "{$account1->getOwnerName()}の口座残高: " . number_format($account1->getBalance()) . "円\n";
echo "{$account2->getOwnerName()}の口座残高: " . number_format($account2->getBalance()) . "円\n";

echo "\n=== 口座照会書 ===\n";
echo $account1->getStatement();

?>

### 解説

#### 1. 取引履歴の管理

**全ての操作を記録**:
```php
private function recordTransaction(Transaction $transaction): void {
    $this->transactions[] = $transaction;
}
```

**ポイント**:
- 各操作の直後に履歴を記録
- 操作後の残高も保存
- DateTimeで正確な時刻を記録

#### 2. バリデーション

**出金時のチェック**:
```php
public function withdraw(int $amount, string $description = "出金"): bool {
    if ($amount <= 0 || $amount > $this->balance) {
        return false;
    }
    // ...
}
```

**チェック項目**:
1. 金額が正の数であること
2. 残高が不足していないこと
3. 操作が成功したかを返す

#### 3. 複数のオブジェクト連携

**振込メソッド**:
```php
public function transfer(BankAccount $toAccount, int $amount, string $description): bool {
    if (!$this->withdraw($amount, $description)) {
        return false;
    }
    $toAccount->deposit($amount, $description);
    return true;
}
```

**ポイント**:
- 別のオブジェクトのメソッドを呼び出す
- 失敗時のロールバック（withdrawが失敗したらdepositしない）
- 双方の履歴に記録される

#### 4. Web開発での応用

**決済システム**:
```php
class PaymentGateway {
    public function charge(CreditCard $card, int $amount): PaymentResult {
        // 決済処理
    }
}

class Order {
    public function pay(PaymentMethod $method): bool {
        // 支払い処理
    }
}
```

---
## ベストプラクティスまとめ

### 1. クラス設計の原則

**✓ やるべきこと**:
- 単一責任の原則（1クラス=1役割）
- 状態をカプセル化（privateプロパティ）
- メソッドで操作を提供
- 明確なインターフェース

**× やるべきでないこと**:
- プロパティをpublicにする（原則）
- 1つのクラスに多くの責任を持たせる
- 状態を直接外部から変更させる

### 2. コンストラクタのベストプラクティス

```php
// 良い例
public function __construct(
    private string $name,  // プロパティプロモーション（PHP 8.0+）
    private int $age
) {}

// 従来の書き方
public function __construct(string $name, int $age) {
    $this->name = $name;
    $this->age = $age;
}
```

### 3. 値のバリデーション

**セッター/メソッド内でチェック**:
```php
public function setAge(int $age): void {
    if ($age < 0 || $age > 150) {
        throw new InvalidArgumentException("無効な年齢");
    }
    $this->age = $age;
}
```

### 4. 状態管理のパターン

| パターン | 説明 | 例 |
|----------|------|-----|
| ステートマシン | 状態遷移を明確に | 貸出中→利用可能 |
| フラグ | 真偽値で状態表現 | isAlive, isBorrowed |
| カウンター | 数値で状態表現 | hp, balance |

### 5. 実践的な開発例

**Eloquentモデル（Laravel）の設計**:
```php
class User extends Model {
    protected $fillable = ['name', 'email'];
    
    public function orders() {
        return $this->hasMany(Order::class);
    }
    
    public function hasOrderedProduct(int $productId): bool {
        return $this->orders()
            ->where('product_id', $productId)
            ->exists();
    }
}
```

### 🎯 次のステップ

第12章のクラスとオブジェクト基礎をマスターしました！次は第13章で、アクセス修飾子とゲッター/セッターについて学びます。

**次章の予習**:
- public, protected, privateの違い
- ゲッターとセッターの使い方
- 静的メソッドと静的プロパティ
- マジックメソッド